In [ ]:
import glob
import numpy as np
import pandas as pd
import os
path =os.path.join('/Volumes','imaging_analysis','2010_08_21_Malaria_MartiLab_MatthiasMarti_HSPH','2016_03_25') # use your path
#get training data for object classes (rbc, ring, gam)
trainData_noB = pd.read_csv(os.path.join(path,'analysis_malaria_2016_03_25_train_Per_FilteredRBC.csv'),index_col=None)
trainData_noB = trainData_noB.sort_values('ImageNumber')

#get training data for background
trainData_B = pd.read_csv(os.path.join(path,'analysis_malaria_2016_03_25_train_B_Per_FilteredBackground.csv'),index_col=None)
trainData_B = trainData_B.sort_values('ImageNumber')

#get test data
testData = pd.read_csv(os.path.join(path,'analysis_malaria_2016_03_25_test_Per_FilteredRBC.csv'),index_col=None)
testData = testData.sort_values(['ImageNumber','FilteredRBC_Number_Object_Number'])


In [ ]:
#get the class information

classes = {'rbc':0, 'ring':1, 'gam':2, '3':3}
#for each file in ImageSets/train.txt, get the annotation file, get the class
input_dir = os.path.join('/Users', 'jyhung', 'Downloads', 'try1', 'data')
input_dir2 = os.path.join('/Volumes','imaging_analysis','2010_08_21_Malaria_MartiLab_MatthiasMarti_HSPH','2016_03_25')
list_train = [] #classes of training set (not including background)
list_test = [] #class of test set
list_train_xy = [] #bounding box coordinates [xmin, ymin, xmax, ymax] of each training set point
list_test_xy = [] #bounding box coordinates [xmin, ymin, xmax, ymax] of each test set point
difficult_index = [] #indices of points tagged as difficult
test_image_number = [] #indices of test images

for train_or_test in ['train', 'test']:
    if train_or_test == 'train':
        filename = os.path.join(input_dir2,'load_data_train_cluster.csv')
        start = 1
    else:
        filename = os.path.join(input_dir, 'ImageSets', train_or_test+'.txt')
        start = 0
    with open(filename) as f:
        im_data = f.readlines()
    num_ims = len(im_data)
    
    # for each image, get per object information
    for ix in range(start,num_ims):
        #get Annotations
        if train_or_test == 'train':
            file_ = im_data[ix].strip().split(',')[1]
            file_subimage_ = file_.split('.')[0].split('_')
            filename_subimage = '_'.join(file_subimage_[0:2])
        else:
            filename_subimage = im_data[ix].strip()
        with open(os.path.join(input_dir, 'Annotations', filename_subimage+'.txt')) as f:
            cell_data = f.readlines()

        #for each object, get location and class information
        if train_or_test == 'train':
            object_range = range(int(file_subimage_[2]),int(file_subimage_[2])+1)
        else:
            object_range = range(len(cell_data))  
        for iix in object_range:
            try:
                x1, y1, x2, y2, cls, difficult = cell_data[iix].strip().split(' ')
            except:
                raise Exception('Error in reading data, line %s:%s'%(str(ix+1), data[ix]))
            if not difficult == 'True':
                if train_or_test == 'train':
                    list_train.append(cls)
                    list_train_xy.append([x1,y1,x2,y2])
                elif train_or_test == 'test':
                    list_test.append(cls)
                    list_test_xy.append([x1,y1,x2,y2])
                    test_image_number.append(ix+1)
            elif train_or_test == 'test':
                    difficult_index.append(iix)
            
trainy_noB = list_train
testy_gt = list_test
test_xy = list_test_xy
